In [23]:
import numpy as np
import pandas as pd
import torch
import torchtext
import torch.nn as nn
from torchtext.legacy import data
import random
import spacy

print(torch.__version__)

1.9.0+cpu


In [4]:
train_df = pd.read_csv("train.csv")

In [5]:
train_df[train_df["target"] == 0]["text"].values[np.random.randint(0, 1000)]

'Casualty Team: Ice Cream Recall Sends Chill Through Food Industry http://t.co/6GsAmY6mts'

In [16]:
train_df[train_df["target"] == 1]["text"].values[np.random.randint(0, 1000)]

'Service on the Green Line has resumed after an earlier derailment near Garfield with residual delays.'

In [171]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

datafields = [(None, None), (None, None), (None, None), ("text", TEXT), ("target", LABEL)]

In [99]:
TrainSet = data.TabularDataset("train.csv", "CSV", datafields, skip_header = True)

In [100]:
print(vars(TrainSet.examples[46]))

{'text': ['How', 'the', 'West', 'was', 'burned', ':', 'Thousands', 'of', 'wildfires', 'ablaze', 'in', 'California', 'alone', 'http://t.co/vl5TBR3wbr'], 'target': '1'}


In [102]:
train_data, valid_data = TrainSet.split(random_state = random.seed(SEED))

In [103]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache\glove.6B.zip: 862MB [53:32, 268kB/s]                                                                     
100%|██████████████████████████████████████████████████████████████████████▉| 399999/400000 [00:17<00:00, 23069.58it/s]


In [104]:
print(len(TEXT.vocab))

20869


In [105]:
print(TEXT.vocab.freqs.most_common(20))

[('#', 2297), ('?', 2124), ('.', 1997), ('the', 1838), (':', 1332), ('a', 1285), ('to', 1260), ('in', 1246), ('of', 1233), ('I', 1081), ('-', 1042), ('and', 904), ("'", 896), ('!', 804), ('...', 676), ('is', 602), ('on', 576), ('for', 571), ("'s", 537), ('you', 522)]


In [106]:
print(LABEL.vocab.stoi)

defaultdict(None, {'0': 0, '1': 1})


In [143]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data), batch_size = BATCH_SIZE, device = device, sort=False)
train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data),
                                                            device = device,
                                                            batch_size = BATCH_SIZE, 
                                                            sort_key = lambda x: len(x.text), 
                                                            sort_within_batch = True)

In [122]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [123]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [124]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,397,557 trainable parameters


In [125]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

model.embedding.weight.data.copy_(pretrained_embeddings)

torch.Size([20869, 100])


tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [ 0.6000, -0.4374,  0.9113,  ..., -0.1978, -0.7155,  0.7696],
        [-1.1576,  1.1766, -1.8410,  ...,  0.2028, -0.0793, -0.2130],
        [ 0.3173,  0.2025, -0.1243,  ...,  0.5444,  0.4876, -0.1643]])

In [126]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [ 0.6000, -0.4374,  0.9113,  ..., -0.1978, -0.7155,  0.7696],
        [-1.1576,  1.1766, -1.8410,  ...,  0.2028, -0.0793, -0.2130],
        [ 0.3173,  0.2025, -0.1243,  ...,  0.5444,  0.4876, -0.1643]])


In [127]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [128]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [146]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.target)
        
        acc = binary_accuracy(predictions, batch.target)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [145]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.target)
            
            acc = binary_accuracy(predictions, batch.target)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [131]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [147]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 43s
	Train Loss: 0.607 | Train Acc: 66.64%
	 Val. Loss: 0.527 |  Val. Acc: 75.00%
Epoch: 02 | Epoch Time: 0m 43s
	Train Loss: 0.533 | Train Acc: 73.32%
	 Val. Loss: 0.532 |  Val. Acc: 74.87%
Epoch: 03 | Epoch Time: 0m 43s
	Train Loss: 0.493 | Train Acc: 76.33%
	 Val. Loss: 0.508 |  Val. Acc: 76.54%
Epoch: 04 | Epoch Time: 0m 43s
	Train Loss: 0.458 | Train Acc: 79.08%
	 Val. Loss: 0.470 |  Val. Acc: 78.07%
Epoch: 05 | Epoch Time: 0m 43s
	Train Loss: 0.418 | Train Acc: 81.56%
	 Val. Loss: 0.523 |  Val. Acc: 77.58%


In [149]:
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [150]:
predict_sentiment(model, "Shots heard near from a school")

0.7335058450698853

In [152]:
predict_sentiment(model, "Do you know the way")

0.04217509552836418

In [170]:
predict_sentiment(model, "")

0.42096850275993347